In [2]:
!pip install careerjet_api_client

  Created wheel for careerjet-api-client: filename=careerjet_api_client-3.0.1-py3-none-any.whl size=4259 sha256=42e90c71091e2897c764d559ed566e9dbf0213aad0141ec93cd0289f494a5522
  Stored in directory: c:\users\mahfoudh\appdata\local\pip\cache\wheels\50\48\cb\a4ce98e9b5be16fbcfe49465cf438cd87aae054cd6ad17db32
Successfully built careerjet-api-client


In [20]:
from careerjet_api_client import CareerjetAPIClient
import pandas as pd
import argparse
import urllib

In [21]:
# Function to make api call
def make_request(page, args):
    url = add_url_params(args)
    cj = CareerjetAPIClient("en_US");
    result_json = cj.search({
        'location': args.location,
        'affid': args.aff_id,
        'user_ip': args.user_ip,
        'pagesize': args.page_size,
        'keywords': args.keywords,
        'page': page,
        'url': url,
        'user_agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:31.0) Gecko/20100101 Firefox/31.0'
    });
    return result_json['pages'], result_json


In [39]:
# searching jobs available by loction and keywords
def request_jobs(args):
    # Getting total pages number
    if (not args.pages):
        pages, _ = make_request(1, args)
    else:
        pages = args.pages

    # getting jobs in each page
    job_list = []
    for page_number in range(pages):
        _, result_json = make_request(page_number, args)
        for job in result_json["jobs"]:
            job_list.append(job)

    # Converting list of jobs to dataframe
    df = pd.DataFrame(job_list)
    
    # Storing data into csv file
    df.to_csv(args.csv_destination)
    
    # Returning dataframe
    return df

In [40]:
# A function that take keywords, and converts it into url parameters
def add_url_params(args):
    param_dict = {}
    param_dict["s"] = args.keywords
    param_dict["l"] = args.location
    url = args.api_url + '?' + urllib.parse.urlencode(param_dict)
    return url

In [45]:
# Main function
def main(args):
    parser = argparse.ArgumentParser(description='Controling search keywords and parameters')
    parser.add_argument(
        '-k', '--keywords', required=False, type=str, help='keywords used in searching', default="python automation")
    parser.add_argument(
        '-p', '--pages', required=False, type=int, help='Number of pages')
    parser.add_argument(
        '-ps', '--page_size', required=False, type=int, help='size of jobs to be extracted on each request', default=99)
    parser.add_argument(
        '-l', '--location', required=False, type=str, help='location of searched jobs', default="New York City, NY")
    parser.add_argument(
        '-ai', '--aff_id', required=False, type=str, help='API id provided by careetjet', default="213e213hd12344552")
    parser.add_argument(
        '-i', '--user_ip', required=False, type=str, help="user's ip", default="11.22.33.44")
    parser.add_argument(
        '-u', '--api_url', required=False, type=str,
        help="url of the request, can be found when doing a search in careerjet",
        default="https://www.careerjet.com/search/jobs")
    parser.add_argument(
        '-c', '--csv_destination', required=False, type=str, help="user's ip", default="jobs_list.csv")

    # Getting command line arguments
    args = parser.parse_args(args)

    # searching jobs
    df = request_jobs(args)
    print(df.head())

In [46]:
# Define your arguments here (same as command line arguments, but this time assign it to the args variable)
args = ""
if __name__ == '__main__':
    main(args)

           locations site                           date  \
0  New York City, NY       Wed, 27 Jan 2021 23:15:47 GMT   
1  New York City, NY       Tue, 26 Jan 2021 23:06:37 GMT   
2  New York City, NY       Tue, 26 Jan 2021 23:34:21 GMT   
3  New York City, NY       Fri, 22 Jan 2021 23:59:31 GMT   
4  New York City, NY       Fri, 22 Jan 2021 23:01:58 GMT   

                                                 url  \
0  http://jobviewtrack.com/en-us/job-1f1b417e480a...   
1  http://jobviewtrack.com/en-us/job-191d416c5810...   
2  http://jobviewtrack.com/en-us/job-1c1e416c5810...   
3  http://jobviewtrack.com/en-us/job-1c4d417e480a...   
4  http://jobviewtrack.com/en-us/job-481d417d5410...   

                                             title  \
0  Senior Python Developer - Investment Management   
1   Cloud Automation Engineer (No C2C/Sponsorship)   
2                   Infrastructure Automation Lead   
3    Senior Software Engineer - BQ Test Automation   
4                          Quant